In [49]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

In [17]:
#Parameters 
x = 66 # Starting Age
alpha = 0.019 # Aging in Place
theta = 0.70 # Non-durable cons exp. share
epsilon = 1.25 # Intratemporal elasticity of subs.
sigma = 0.333 # Intertemporal elasticity of subs.
kappa = 6000 # Bequest motive curvature
b = 12 # Bequest motive intensity

In [50]:
ir = np.load('ir_annual_.npy')
dvmr = np.load('dvmr_annual_.npy')
hp =  np.load('house_annual_.npy')
cpi = np.load('cpi_annual_.npy')
df_hc  = pd.read_csv('df_hc.csv')

In [51]:
df_hc

,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,...,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057
0,2092.244113,2161.960451,2208.952751,2268.076710,2332.292630,2395.375061,2459.935522,2527.402279,2596.593673,2667.184149,...,4101.747954,4213.581151,4328.463537,4446.478022,4567.710117,4692.247638,4820.180651,4951.601699,5086.605910,5225.290988
1,2092.244113,2161.960451,2208.952751,2268.076710,2332.292630,2395.375061,2459.935522,2527.402279,2596.593673,2667.184149,...,4101.747954,4213.581151,4328.463537,4446.478022,4567.710117,4692.247638,4820.180651,4951.601699,5086.605910,5225.290988
2,2092.244113,2161.960451,2208.952751,2268.076710,2332.292630,2395.375061,2459.935522,2527.402279,2596.593673,2667.184149,...,4101.747954,4213.581151,4328.463537,4446.478022,4567.710117,4692.247638,4820.180651,4951.601699,5086.605910,5225.290988
3,2092.244113,2161.960451,2208.952751,2268.076710,2332.292630,2395.375061,2459.935522,2527.402279,2596.593673,2667.184149,...,4101.747954,4213.581151,4328.463537,4446.478022,4567.710117,4692.247638,4820.180651,4951.601699,5086.605910,5225.290988
4,1941.482304,2007.581179,2052.648722,2109.052256,2170.264193,2230.496938,2292.181290,2356.651386,2422.809707,2490.355062,...,3869.763849,3977.752517,4088.743298,4202.819192,4320.065821,4440.571146,4564.425428,4691.721441,4822.554584,4957.022879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11507.342620,11890.782483,12149.240131,12474.421904,12827.609467,13174.562838,13529.645372,13900.712535,14281.265204,14669.512821,...,22067.908307,22633.725766,23214.135737,23809.515147,24420.252565,25046.746523,25689.405161,26348.647352,27024.903189,27718.613892
96,11507.342620,11890.782483,12149.240131,12474.421904,12827.609467,13174.562838,13529.645372,13900.712535,14281.265204,14669.512821,...,22257.025784,22827.622025,23412.932287,24013.336591,24629.226696,25261.004406,25909.081208,26573.879411,27255.832634,27955.385712
97,11507.342620,11890.782483,12149.240131,12474.421904,12827.609467,13174.562838,13529.645372,13900.712535,14281.265204,14669.512821,...,22446.143262,23021.518284,23611.728836,24217.158034,24838.200827,25475.262289,26128.757256,26799.111471,27486.762079,28192.157532
98,11507.342620,11890.782483,12149.240131,12474.421904,12827.609467,13174.562838,13529.645372,13900.712535,14281.265204,14669.512821,...,22559.613748,23174.696329,23806.549455,24420.979477,25047.174958,25689.520172,26348.433303,27024.343530,27717.691524,28428.929352


In [9]:
def omega(t,homeowner=True):
    
    if homeowner:
        A_t = t - x
        
    else:
        A_t = 0
        
    omega_t = np.exp(alpha * A_t)
    
    return omega_t


In [15]:
def U1(C_t, S_t,t,homeowner):
    omega_t = omega(t, homeowner)
    
    term1 = theta**(1 / epsilon) * C_t**((epsilon - 1) / epsilon)
    
    term2 = (1 - theta)**(1 / epsilon) * (omega_t * S_t)**((epsilon - 1) / epsilon)
    
    utility = ((term1 + term2)**(epsilon / (epsilon - 1)))**(1 - (1 / sigma)) / (1 - (1 / sigma))
    
    return utility

In [12]:
def U2(W_tau_x):
    
    utility = b * (W_tau_x + kappa)**(1 - (1 / sigma)) / (1 - (1 / sigma))
    
    return utility

In [47]:
def W_next(W_t, C_t, HC_t, HE_t, r_t, Y_t, BAL_t, pi_m):
    W_t_plus_1 = (W_t - C_t - HC_t - HE_t - pi_m * BAL_t) * (1 + r_t) + Y_t
    return W_t_plus_1

NameError: name 'df_hc' is not defined